In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [35]:
browser=webdriver.Chrome()

In [36]:
browser.get('https://www.tripadvisor.in/Hotels-g186338-a_ufe.true-London_England-Hotels.html')


In [37]:
accept = browser.find_element(By.ID, "onetrust-accept-btn-handler")
accept.click()

In [38]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [39]:
list_of_numbers = str(list(range(1, 101)))
hotels = []
for name in soup.findAll('a',{'data-clicksource':'HotelName'}):
      hotels.append(name.text.strip(list_of_numbers + '.'))
hotels[:5]


['YOTEL London Shoreditch',
 'The Montcalm Royal London House',
 'Park Grand London Hyde Park',
 'The Resident Covent Garden',
 'Park Grand Paddington Court']

In [40]:
ratings = []
for rating in soup.findAll('a',{'class':'ui_bubble_rating'}):
      if rating['alt'] == "5 of 5 bubbles":
            rating = 10
      elif rating['alt'] == "4.5 of 5 bubbles":
            rating = 9
      elif rating['alt'] == "4 of 5 bubbles":
            rating = 8
      elif rating['alt'] == "3.5 of 5 bubbles":
            rating = 7
      elif rating['alt'] == "3 of 5 bubbles":
            rating = 6
      else:
            rating = 5
      ratings.append(rating)
len(ratings)

34

In [41]:
reviews = []
for review in soup.findAll('a',{'class':'review_count'}):
  reviews.append(review.text.replace('reviews','').replace(',','').strip())
len(reviews)

34

In [42]:
price = []
for p in soup.findAll('div',{'data-sizegroup':'mini-meta-price'}):
  price.append(p.text.replace('€','').replace(',','').strip()) 
price[:5]
len(price)

34

In [43]:
d1 = {'Hotel':hotels,'Ratings':ratings,'No_of_Reviews':reviews,'Price, €':price}
df = pd.DataFrame.from_dict(d1)

In [44]:
df

Hotel  Ratings No_of_Reviews  \
0                         YOTEL London Shoreditch        9           685   
1                 The Montcalm Royal London House       10          3354   
2                     Park Grand London Hyde Park        8          3362   
3                      The Resident Covent Garden       10          1244   
4                     Park Grand Paddington Court        9          3212   
5                    Travelodge London City hotel        9          2378   
6      The Biltmore Mayfair, LXR Hotels & Resorts        8           173   
7                    Canopy by Hilton London City       10           424   
8                 The Montcalm London Marble Arch       10          3812   
9                          Hard Rock Hotel London        9          2244   
10                    The Clermont, Charing Cross       10          6724   
11                                The Tower Hotel        8         12243   
12                    Novotel London Tower Bridge        8          3709   
13            Travelodge London Docklands Central        8           576   
14         Leonardo Royal Hotel London St. Paul's        8          1457   
15             Leonardo Royal London Tower Bridge        8          2147   
16                      Holiday Inn London - West        9          1281   
17                           Royal National Hotel        6         10766   
18           Park Plaza Westminster Bridge London        9         23721   
19            Travelodge London Central City Road        8          2198   
20                    The Montague on The Gardens       10          5449   
21                           The Standard, London        9           495   
22  Millennium Gloucester Hotel London Kensington        8          6715   
23                        The Resident Kensington        9          4753   
24                  Travelodge London Manor House        9           264   
25           The Bailey's Hotel London Kensington        9          1923   
26                                  The Clarendon        8          2944   
27                Travelodge London Covent Garden        8         12308   
28                  Grand Royale London Hyde Park        8          5417   
29                            The President Hotel        7          4704   
30     Premier Inn London Hendon (The Hyde) hotel        8           842   
31                               St. Athans Hotel        8           855   
32                            Novotel London West        8          5326   
33                          Mercure London Bridge        9           440   

   Price, €  
0       112  
1       207  
2       166  
3       302  
4       145  
5        59  
6       930  
7       194  
8       473  
9       386  
10      385  
11      156  
12      175  
13       74  
14      214  
15      178  
16       99  
17      217  
18      301  
19       65  
20      347  
21      306  
22      137  
23      194  
24       96  
25      233  
26      154  
27       91  
28      139  
29      193  
30       82  
31      109  
32      164  
33      188

In [45]:
df.to_csv("tripadvisor_london_list.csv")

In [46]:
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

In [47]:
tripadvisor = pd.read_csv('tripadvisor_london_list.csv')
tripadvisor.head()


Unnamed: 0                            Hotel  Ratings  No_of_Reviews  \
0           0          YOTEL London Shoreditch        9            685   
1           1  The Montcalm Royal London House       10           3354   
2           2      Park Grand London Hyde Park        8           3362   
3           3       The Resident Covent Garden       10           1244   
4           4      Park Grand Paddington Court        9           3212   

   Price, €  
0       112  
1       207  
2       166  
3       302  
4       145

In [48]:
tripadvisor.to_sql('tripadvisor', conn, if_exists='append', index = False)


34

In [49]:
booking = pd.read_csv('bookingcom_london_list.csv')
booking.head()

Unnamed: 0                       Hotel  Ratings  No_of_Reviews  Price, €
0           0             Fitzrovia Hotel      7.8           2624       221
1           1        ApartaSuites Emerald      8.2              4       171
2           2  The Windmill On The Common      8.5            767       146
3           3              The Kings Arms      9.4            612       204
4           4        The Tommyfield Hotel      8.6            483       135

In [50]:
booking.to_sql('booking_com', conn, if_exists='append', index = False)

25